In [ ]:
# Importing Dependencies

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import numpy as np
import pandas as pd
import glob
import xarray as xr
import warnings
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import rioxarray
from rasterio import features
from affine import Affine
import datetime

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from pyhdf.SD import SD, SDC

plt.style.use('ggplot')

warnings.filterwarnings('ignore')

# LOAD INSAT DATA

In [ ]:
# TESTING ON ACTUAL DATA
#Loading AOD Dataset for testing 
aod_file = glob.glob('./Data/AOD/*')
aod_insat3D = glob.glob(aod_file[1]+'/*')

aod_2015 = []
aod_JFM_2014 = []
for file in aod_insat3D:
    file_date = file.split('\\')[-1].split('_')[1]
    
    month = file_date[2:5]
    year = file_date[5:]
    
    if (month == 'JAN' or month == 'FEB' or month == 'MAR') and year == '2014':
        aod_JFM_2014.append(file)
    if year == '2015':
        aod_2015.append(file)
    
datasets = []
for file in aod_2015:
    ds = xr.open_dataset(file)
    # Sort the longitude coordinate in ascending order
    ds = ds.sortby('longitude').sortby('latitude')
    ds = ds.sel(latitude=slice(8,38),longitude=slice(68,98))
    #print(ds.dims)
    datasets.append(ds)

ds_concat = xr.concat(datasets, dim='time')
ds = xr.open_mfdataset(aod_JFM_2014,combine="nested",concat_dim='time')
ds_temp = ds.sel(latitude=slice(8,38),longitude=slice(68,98)).load()

In [ ]:
fig,axes =  plt.subplots(1,2,figsize=(13,4))
ds_temp.AOD.isel(time=0).plot(ax=axes[0])
ds_temp.AOD.isel(time=1000).plot(ax=axes[1])
plt.show()

# Load Aeronet Data

In [1]:
def read_aeronet_data(path):  
    df_temp = pd.read_csv(path,sep='\t',skiprows=[0])
    #Get lat and lon
    lat_loc,lon_loc = df_temp['Site_Latitude(Degrees)'].unique(), df_temp['Site_Longitude(Degrees)'].unique()
      
    
    # Taking only date and aod column
    df_temp = df_temp[['Date(dd:mm:yyyy)','AOD_675nm',]]
    
    df_temp['Date'] = pd.to_datetime(df_temp['Date(dd:mm:yyyy)'], format='%d:%m:%Y')
    df_temp = df_temp.drop(columns=['Date(dd:mm:yyyy)'])
    df_temp = df_temp.replace(-999, np.nan)
    df_temp = df_temp.sort_values(by='Date')
    
    df_temp = df_temp.rename(columns={'Date': 'time'})
    return df_temp,lat_loc,lon_loc

In [2]:
# AERONET DATASET
aeronet_dataset = glob.glob(aod_file[3]+'/*')

# LOAD AERONET DATASET
print(aeronet_dataset)


gual_path = str(glob.glob(aeronet_dataset[0]+'/*')[0])

pune_path = str(glob.glob(aeronet_dataset[1]+'/*')[0])

gandhi_clg = str(glob.glob(aeronet_dataset[2]+'/*')[0])

jaipur_path = str(glob.glob(aeronet_dataset[3]+'/*')[0])

kanpur_path = str(glob.glob(aeronet_dataset[4]+'/*')[0])

df,lat_loc,lon_loc = read_aeronet_data(jaipur_path)

print(lat_loc,lon_loc)
df.head()

NameError: name 'glob' is not defined

In [ ]:
# Select time period
start_date = '2015-01-01'
end_date = '2015-12-31'
insat_data = ds_concat.sel(time=slice(start_date,end_date)).sortby('time')
aeronet_data = df.loc[(df['time'] >= start_date) & (df['time'] <= end_date)]


def convert_date(df,f = '1D'):
    # set 'Date' column as the index
    temp = df.set_index('time')

    # group data by time and calculate the mean 'AOD' value
    temp_df = temp.groupby(pd.Grouper(freq=f)).mean()
    # reset the index to include the 'Date' column

    temp_df = temp_df.reset_index()

    return temp_df

# select location
insat_data = insat_data.sel(latitude=lat_loc,longitude=lon_loc,method='nearest')

# daily
insat_data = insat_data.resample(time='1D').mean().sortby('time')
aeronet_data = convert_date(aeronet_data,f='1D')

In [ ]:
insat_data

In [ ]:
aeronet_data

In [ ]:
#insat_data.AOD.values

# Scatter plot

In [ ]:
from sklearn.linear_model import LinearRegression

# scatter plot
insat_df = insat_data.to_dataframe().reset_index()
#modis_df = modis_data.to_dataframe().reset_index()


# Merge the INSAT and MODIS dataframes by time
merged_df = pd.merge(aeronet_data,insat_df, on='time')

merged_df = merged_df.drop(['latitude', 'longitude'], axis=1)
merged_df = merged_df.dropna(subset=['AOD_675nm', 'AOD'], how='any')

merged_df

# Create the scatter plot
fig, axes = plt.subplots(1,2,figsize=(15,5))
ax = axes[0]
ax.scatter(merged_df['AOD_675nm'], merged_df['AOD'], color='blue')
ax.set_xlabel('AERONET AOD 675nm')
ax.set_ylabel('INSAT AOD 650nm')

# Calculate the regression line
regression = LinearRegression().fit(merged_df['AOD_675nm'].values.reshape(-1, 1), merged_df['AOD'].values.reshape(-1, 1))
slope = regression.coef_[0][0]
intercept = regression.intercept_[0]

# Calculate the limits
limit = max(merged_df['AOD_675nm'].max(), merged_df['AOD'].max())

# Add the regression line to the scatter plot
x_vals = np.arange(0,limit+2)
y_vals = intercept + slope * x_vals
ax.plot(x_vals, y_vals, '--', color='blue')

# RMSE and Corr
rmse = np.sqrt(np.mean((merged_df['AOD_675nm'] - merged_df['AOD']) ** 2))
corr_coef = np.corrcoef(merged_df['AOD_675nm'], merged_df['AOD'])[0,1]

# Add RMSE and correlation coefficient as text inside the plot
textstr = f'RMSE = {rmse:.2f}\nR = {corr_coef:.2f}'
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)


# Calculate the step
step = 0.5

# Set the limits and step to be the same for both axes
ax.set_xlim([0, limit])
ax.set_ylim([0, limit])
ax.set_xticks(np.arange(0, limit+step, step))
ax.set_yticks(np.arange(0, limit+step, step))
ax.plot(ax.get_xlim(),ax.get_xlim(), '--', color='black')

ax = axes[1]
merged_df.plot(x='time', y=['AOD', 'AOD_675nm'],label=['INSAT','Aeronet'], ax=ax)

#plt.savefig('./plots/aod_plot/scatter_plot/kanpur_2015_scatter')
# Display the plot

plt.suptitle('kanpur,2015',weight='bold')
plt.show()